## 2. Tensorflow Low Level API

### 2.2. 张量Tensors

Tensorflow是定义和执行张量计算的一个框架。张量是对矢量和矩阵向潜在的更高维度的泛化。TensorFlow在内部将张量表示为基本数据类型的n维数组。

张量与张量值的定义：

1. 张量Tensor是操作(op)的输出的一个符号句柄symbolic handle。张量Tensor不包含操作(op)的输出值，只提供在Tensorflow`tf.Session`中计算输出值的方法。

2. 张量值为numpy数组(基本数据类型的n维数组)；

张量Tensor类有两个目的：

1. 张量`Tensor`可以作为参数传给另一个操作(op)的输入，这在操作(op)之间创建数据流连接，使得Tensorflow可以执行整个表示a large, multi-step计算的计算流图；
2. 在session中执行计算流图之后，可以将张量`Tensor`传递给`tf.Session.run`来计算张量值。

张量Tensor属性：

1. `device`: 生成此张量的设备名，或`None`；
2. **`dtype`**: 元素数据类型；
3. `graph`: 包含此张量的计算流图对象`Graph`；
4. `name`: 张量的名字字符串；
5. `op`: 以该张量为输出的操作对象`Operation`；
6. **`shape`**: 返回表示张量形状的`TensorShape`对象；
7. `value_index`: 此张量在操作对象`Operation`的输出索引；

>Ps: 张量的dtype一定是已知的。shape，即张量的维数和每个维度的大小，可能只有部分已知。

特殊类型张量：

+ [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)
+ [tf.constant](https://www.tensorflow.org/api_docs/python/tf/constant)
+ [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder)
+ [tf.SparseTensor](https://www.tensorflow.org/api_docs/python/tf/SparseTensor)

除了[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)以外，张量的值是不可变的，即对于单次执行`tf.Session.run`，张量只有一个值。


**张量的阶rank：张量的维数**

---

注意：张量的阶rank不同于矩阵的秩rank

Rank | Math entity
------ | ----------
0 | Scalar (magnitude only)
1 | Vector (magnitude and direction)
2 | Matrix (table of numbers)
3 | 3-Tensor (cube of numbers)
n | n-Tensor (自行想象)


**0阶张量**

0阶张量创建方法如下，张量将字符串视作单一元素，而不是一连串字符元素：

In [3]:
import tensorflow as tf

mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

**1阶张量**

传递一个列表作为输入值来创建1阶张量，例如：

In [4]:
mystr = tf.Variable(['Hello'], tf.string)  # 注意传入的是字符串的列表
cool_numbers = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.13j], tf.complex64)

**更高阶张量**

2阶张量至少包含一行和一列：

In [5]:
mymat = tf.Variable([[7], [11]], tf.int16)
myxor = tf.Variable([[False, True], [True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([[4, 9], [16, 25]], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
print(rank_of_squares)
mymatC = tf.Variable([[7], [11]], tf.int32)

Tensor("Rank:0", shape=(), dtype=int32)


同样，更高阶的张量由一个 n 维数组组成。例如，在图像处理过程中，会使用许多 4 阶张量，维度对应批大小batch size、图像宽度、图像高度和颜色通道。

In [6]:
my_image = tf.zeros([10, 299, 299, 3])  # batch * height * width * color

**获取[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)对象的阶**

使用[tf.rank](https://www.tensorflow.org/api_docs/python/tf/rank)方法获取[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)对象的阶：

In [7]:
r = tf.rank(my_image)
print(r)

# After the graph runs, r will hold the value 4.

Tensor("Rank_1:0", shape=(), dtype=int32)


**[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn)张量索引**

---

张量[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn)是n维单元数组，需要指定n个索引，索引可以是int或者0阶张量；

1. 正负索引：索引单个元素
2. 切片索引：n个切片索引返回与原张量阶数相等的子张量；` : `是Python切片语法，意味“该维度不变”。
3. 正负 + 切片索引：返回比原张量阶数小的子张量；



```
 |  __getitem__ = _slice_helper(tensor, slice_spec, var=None)
 |      Overload for Tensor.__getitem__.
 |
 |      This operation extracts the specified region from the tensor.
 |      The notation is similar to NumPy with the restriction that
 |      currently only support basic indexing. That means that
 |      using a non-scalar tensor as input is not currently allowed.
 |
 |      Some useful examples:
 |
 |      ``python
 |      # strip leading and trailing 2 elements
 |      foo = tf.constant([1,2,3,4,5,6])
 |      print(foo[2:-2].eval())  # => [3,4]
 |
 |      # skip every row and reverse every column
 |      foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
 |      print(foo[::2,::-1].eval())  # => [[3,2,1], [9,8,7]]
 |
 |      # Use scalar tensors as indices on both dimensions
 |      print(foo[tf.constant(0), tf.constant(2)].eval())  # => 3
 |
 |      # Insert another dimension
 |      foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
 |      print(foo[tf.newaxis, :, :].eval()) # => [[[1,2,3], [4,5,6], [7,8,9]]]
 |      print(foo[:, tf.newaxis, :].eval()) # => [[[1,2,3]], [[4,5,6]], [[7,8,9]]]
 |      print(foo[:, :, tf.newaxis].eval()) # => [[[1],[2],[3]], [[4],[5],[6]],
 |      [[7],[8],[9]]]
 |
 |      # Ellipses (3 equivalent operations)
 |      foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
 |      print(foo[tf.newaxis, :, :].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
 |      print(foo[tf.newaxis, ...].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
 |      print(foo[tf.newaxis].eval())  # => [[[1,2,3], [4,5,6], [7,8,9]]]
 |      ``
 |
 |      Notes:
 |        - `tf.newaxis` is `None` as in NumPy.
 |        - An implicit ellipsis is placed at the end of the `slice_spec`
 |        - NumPy advanced indexing is currently not supported.
 |
 |      Args:
 |        tensor: An ops.Tensor object.
 |        slice_spec: The arguments to Tensor.__getitem__.
 |        var: In the case of variable slice assignment, the Variable
 |          object to slice (i.e. tensor is the read-only view of this
 |          variable).
 |
 |      Returns:
 |        The appropriate slice of "tensor", based on "slice_spec".
 |
 |      Raises:
 |        ValueError: If a slice range is negative size.
 |        TypeError: If the slice indices aren't int, slice, or Ellipsis.
 ```

In [8]:
with tf.Session() as sess:

    # strip leading and trailing 2 elements
    foo = tf.constant([1, 2, 3, 4, 5, 6])
    print(foo[2:-2].eval())  # print [3, 4]

    # skip second row and reverse every column
    foo = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    print(foo[::2, ::-1].eval())  # print [[3, 2, 1], [9, 8, 7]]

    # Use scalar tensors as indices on both dimensions
    print(foo[tf.constant(0), tf.constant(2)].eval())  # print 3

    # Insert another dimension
    foo = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    print(foo[tf.newaxis, :, :].eval())  # print [[[1,2,3], [4,5,6], [7,8,9]]]
    print(foo[:, tf.newaxis, :].eval())  # print [[[1,2,3]], [[4,5,6]], [[7,8,9]]]
    print(foo[:, :, tf.newaxis].eval())  # print [[[1],[2],[3]], [[4],[5],[6]], [[7],[8],[9]]]

    # 省略Ellipses(3 equivalent operations)
    foo = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
    print(foo[tf.newaxis, :, :].eval())  # print [[[1,2,3], [4,5,6], [7,8,9]]]
    print(foo[tf.newaxis, ...].eval())  # print [[[1,2,3], [4,5,6], [7,8,9]]] 使用了省略对象
    print(foo[tf.newaxis].eval())  # print [[[1,2,3], [4,5,6], [7,8,9]]]

[3 4]
[[3 2 1]
 [9 8 7]]
3
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]]

 [[4 5 6]]

 [[7 8 9]]]
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]

 [[7]
  [8]
  [9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]


**张量的形状shape：每个维度的元素个数**

---

**TensorFlow在图的构建过程中自动推理张量形状。这些推理的形状可能具有已知或未知的阶。如果阶已知，则每个维度的大小可能已知或未知。**TensorFlow automatically infers shapes during graph construction. These inferred shapes might have known or unknown rank. If the rank is known, the sizes of each dimension might be known or unknown.

**张量的形状shape可以使用整型Python列表/元组或者[tf.TensorShape](https://www.tensorflow.org/api_docs/python/tf/TensorShape?hl=zh-cn)表示。**


获取张量形状：

1. 张量shape属性；
2. 将张量shape作为参数来构造另一个张量，在执行图时，新张量将获得原张量的完全定义形状；

```
 |  shape
 |      Returns the `TensorShape` that represents the shape of this tensor.
 |
 |      The shape is computed using shape inference functions that are
 |      registered in the Op for each `Operation`.  See
 |      `tf.TensorShape`
 |      for more details of what a shape represents.
 |
 |      The inferred shape of a tensor is used to provide shape
 |      information without having to launch the graph in a session. This
 |      can be used for debugging, and providing early error messages. For
 |      example:
 |
 |      ``python
 |      c = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
 |
 |      print(c.shape)
 |      ==> TensorShape([Dimension(2), Dimension(3)])
 |
 |      d = tf.constant([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0]])
 |
 |      print(d.shape)
 |      ==> TensorShape([Dimension(4), Dimension(2)])
 |
 |      # Raises a ValueError, because `c` and `d` do not have compatible
 |      # inner dimensions.
 |      e = tf.matmul(c, d)
 |
 |      f = tf.matmul(c, d, transpose_a=True, transpose_b=True)
 |
 |      print(f.shape)
 |      ==> TensorShape([Dimension(3), Dimension(4)])
 |      ``
 |
 |      In some cases, the inferred shape may have unknown dimensions. If
 |      the caller has additional information about the values of these
 |      dimensions, `Tensor.set_shape()` can be used to augment the
 |      inferred shape.
 |
 |      Returns:
 |        A `TensorShape` representing the shape of this tensor.
```


In [10]:
c = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(c.shape, type(c.shape))

d = tf.constant([[1., 0.], [0., 1.], [1., 0.], [0., 1.]])
print(d.shape, type(d.shape))

# Raise a ValueError, because `c` and `d` do not have compatible
# inner dimensions.
# e = tf.matmul(c, d)

f = tf.matmul(c, d, transpose_a=True, transpose_b=True)
print(f.shape, type(f.shape))


(2, 3) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
(4, 2) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
(3, 4) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>


For example, here is how to make a vector of zeros with the same size as the number of columns in a given matrix:

In [15]:
zeros = tf.zeros(f.shape[1])

with tf.Session() as sess:
    print(zeros.eval())

[0. 0. 0. 0.]


改变张量形状--[tf.reshape](https://www.tensorflow.org/api_docs/python/tf/reshape?hl=zh-cn):

张量的**元素数量**是其形状中各维长度的乘积。改变张量形状，张量元素数量不变。

In [16]:
rank_three_tensor = tf.ones([3, 4, 5])
matrix = tf.reshape(rank_three_tensor, [6, 10])  # Reshape existing content into a 6x10 matrix

matrixB = tf.reshape(matrix, [3, -1])  # Reshape existing content into a 3x20 matrix. -1 tells reshape to calculate
                                       # the size of this dimension.
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  # Reshape existing content into a 4x3x5 tensor

# Note that the number of elements of the reshaped Tensors has to match the
# original number of elements. Therefore, the following example generates an
# error because no possible value for the last dimension will match the number
# of elements.
# yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

**数据类型Data types**

---

1. 可以将任意数据结构序列化为string并将其存储在[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)中;
2. [tf.cast](https://www.tensorflow.org/api_docs/python/tf/dtypes/cast)：可以改变[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)的数据类型；
3. TensorFlow将Python整数转型为[tf.int32](https://www.tensorflow.org/api_docs/python/tf/int32?hl=zh-cn)，将python浮点数转型为[tf.float32](https://www.tensorflow.org/api_docs/python/tf/float32?hl=zh-cn)。此外，TensorFlow使用Numpy转换数组的/相同规则来转换张量。

**评估张量Evaluating Tensor**

---

`Tensor.eval`会返回一个与张量内容相同的 NumPy 数组:

```
 |  eval(self, feed_dict=None, session=None)
 |      Evaluates this tensor in a `Session`.
 |
 |      Calling this method will execute all preceding operations that
 |      produce the inputs needed for the operation that produces this
 |      tensor.
 |
 |      *N.B.* Before invoking `Tensor.eval()`, its graph must have been
 |      launched in a session, and either a default session must be
 |      available, or `session` must be specified explicitly.
 |
 |      Args:
 |        feed_dict: A dictionary that maps `Tensor` objects to feed values.
 |          See `tf.Session.run` for a
 |          description of the valid feed values.
 |        session: (Optional.) The `Session` to be used to evaluate this tensor. If
 |          none, the default session will be used.
 |
 |      Returns:
 |        A numpy array corresponding to the value of this tensor.
```

Sometimes it is not possible to evaluate a tf.Tensor with no context because its value might depend on dynamic information that is not available. For example, tensors that depend on placeholders can't be evaluated without providing a value for the placeholder.有时无法在没有背景信息的情况下评估 tf.Tensor，因为它的值可能取决于无法获取的动态信息。例如，在没有为 placeholder 提供值的情况下，无法评估依赖于 placeholder 的张量。

In [21]:
with tf.Session() as sess:
    p = tf.placeholder(tf.float32)
    t = p + 1.0
    # t.eval()  # This will fail, since the placeholder did not get a value.
    print(t.eval(feed_dict={p: 2.0}))  # This will succeed because we're feeding a value to the placeholder

3.0


其他模型构造可能会使评估[tf.Tensor]()变得较为复杂。TensorFlow无法直接评估在函数内部或控制流结构内部定义的[tf.Tensor]()。如果 [tf.Tensor]()取决于队列中的值，那么只有在某个项加入队列后才能评估[tf.Tensor]()；否则，评估将被搁置。在处理队列时，请先调用 [tf.train.start_queue_runners]()，再评估任何[tf.Tensor]()。

[tf.Tensor]: https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn;
[tf.train.start_queue_runners]: https://www.tensorflow.org/api_docs/python/tf/train/start_queue_runners?hl=zh-cn;

**输出张量Printing Tensors**

---

虽然[tfdbg]()提供高级调试支持，但TensorFlow也提供直接输出[tf.Tensor]()值的操作(op)[tf.Print]()。

[tf.Print](): 返回其第一个张量参数（保持不变），同时输出作为第二个参数传入的张量集合。



[tf.Print]: https://www.tensorflow.org/api_docs/python/tf/Print?hl=zh-cn;
[tf.Tensor]: https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn;
[tfdbg]: https://www.tensorflow.org/guide/debugger?hl=zh-cn;

In [27]:
t = tf.constant(3., tf.float32)
tf.Print(t, [t])  # This does nothing
t = tf.Print(t, [t])  # Here we are using the value returned by tf.Print
result = t + 1  # Now when result is evaluated the value of `t` will be printed.

with tf.Session() as sess:
    print(result.eval())

4.0


在评估 result 时，会评估所有影响 result 的元素。由于 result 依靠 t，而评估 t 会导致输出其输入（t 的旧值），所以系统会输出 t。